In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')

In [2]:
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [3]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
delimiters=["।","?","!"]

In [4]:
fasttext=Word2Vec.load('fasttext/hi-d100-m2-fasttext.model')

In [5]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [6]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [7]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [8]:
def word_embedding(word):
    try:
        return fasttext.wv[word]
    except:
        return np.array([0.0]*100)

In [9]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [10]:
def ed(a,b):
    return lin.norm(a-b)

In [11]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [12]:
data=pd.read_csv('../hindi.csv')

In [13]:
query='भौतिक जगत में माया का आधिपत्य होता है और श्रीकृष्ण ने इसे आठवें अध्याय के 15वें श्लोक में अस्थायी और दुखों से भरा हुआ कह कर परिभाषित किया है। इस प्रकार भौतिक शक्ति माया की तुलना अंधकार से की जाती है।'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [14]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [15]:
verse_index=max_cosine(query_embedding)

In [16]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
605,17,17,श्रद्धया परया तप्तं तपस्तत्ित्रविधं नरैः।अफलाक...,"शरीर, वाणी और मन की तपस्याओं का चित्रण करने पश..."
519,14,2,इदं ज्ञानमुपाश्रित्य मम साधर्म्यमागताः।सर्गेऽप...,श्रीकृष्ण अर्जुन को आश्वस्त करते हैं कि वे जो ...
504,13,22,पुरुषः प्रकृतिस्थो हि भुङ्क्ते प्रकृतिजान्गुणा...,पिछले श्लोक में श्रीकृष्ण ने यह व्यक्त किया था...
160,4,4,अर्जुन उवाच\n\nअपरं भवतो जन्म परं जन्म विवस्वत...,अर्जुन भगवान श्रीकृष्ण के कथनों में घटनाओं की ...
607,17,19,मूढग्राहेणात्मनो यत्पीडया क्रियते तपः।परस्योत्...,मूढग्राहेणात्मनो शब्द ऐसे व्यक्तियों के लिए प्...
287,7,13,त्रिभिर्गुणमयैर्भावैरेभिः सर्वमिदं जगत्।\n\nमो...,पिछले श्लोकों को सुनकर अर्जुन यह सोचता होगा “ह...
282,7,8,रसोऽहमप्सु कौन्तेय प्रभास्मि शशिसूर्ययोः।\n\nप...,यह कहने के पश्चात कि वे ही जो सब कुछ दिखाई देत...
273,6,46,तपस्विभ्योऽधिको योगी ज्ञानिभ्योऽपि मतोऽधिकः।\n...,तपस्वी वह है जो स्वेच्छा से वैराग्य स्वीकार कर...
216,5,18,विद्याविनयसंपन्ने ब्राह्मणे गवि हस्तिनि।\n\nशु...,जब हम ज्ञान के परिप्रेक्ष्य में चीजों को देखते...
272,6,45,प्रयत्नाद्यतमानस्तु योगी संशुद्धकिल्बिषः।\n\nअ...,पिछले जन्म के संचित अभ्यास आध्यात्मिक उन्नति क...


## Euclidean Distance

In [17]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [18]:
verse_index=max_ed(query_embedding)

In [19]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
605,17,17,श्रद्धया परया तप्तं तपस्तत्ित्रविधं नरैः।अफलाक...,"शरीर, वाणी और मन की तपस्याओं का चित्रण करने पश..."
519,14,2,इदं ज्ञानमुपाश्रित्य मम साधर्म्यमागताः।सर्गेऽप...,श्रीकृष्ण अर्जुन को आश्वस्त करते हैं कि वे जो ...
607,17,19,मूढग्राहेणात्मनो यत्पीडया क्रियते तपः।परस्योत्...,मूढग्राहेणात्मनो शब्द ऐसे व्यक्तियों के लिए प्...
287,7,13,त्रिभिर्गुणमयैर्भावैरेभिः सर्वमिदं जगत्।\n\nमो...,पिछले श्लोकों को सुनकर अर्जुन यह सोचता होगा “ह...
193,4,37,यथैधांसि समिद्धोऽग्निर्भस्मसात्कुरुतेऽर्जुन।\n...,अग्नि की छोटी-सी चिंगारी भी भयंकर आग का रूप धा...
160,4,4,अर्जुन उवाच\n\nअपरं भवतो जन्म परं जन्म विवस्वत...,अर्जुन भगवान श्रीकृष्ण के कथनों में घटनाओं की ...
504,13,22,पुरुषः प्रकृतिस्थो हि भुङ्क्ते प्रकृतिजान्गुणा...,पिछले श्लोक में श्रीकृष्ण ने यह व्यक्त किया था...
651,18,35,यया स्वप्नं भयं शोकं विषादं मदमेव च।न विमुञ्चत...,अज्ञानी और अहंकारी लोगों में भी दृढ़ता पायी जा...
152,3,39,आवृतं ज्ञानमेतेन ज्ञानिनो नित्यवैरिणा।\n\nकामर...,यहाँ श्रीकृष्ण ने काम या वासना की हानिकारक प्र...
512,13,30,प्रकृत्यैव च कर्माणि क्रियमाणानि सर्वशः।यः पश्...,तंत्र भागवत में वर्णन है-“अहंकारत् तु संसारो भ...


# Mean Pooling

## Cosine Similarity

In [20]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [21]:
verse_index=mean_cosine(query_embedding)

In [22]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
214,5,16,ज्ञानेन तु तदज्ञानं येषां नाशितमात्मनः।\n\nतेष...,रात्रि के अंधकार को मिटाने की सूर्य की शक्ति अ...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
503,13,21,कार्यकारणकर्तृत्वे हेतुः प्रकृतिरुच्यते।पुरुषः...,प्राकृत शक्ति ब्रह्मा के निर्देशानुसार असंख्य ...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
314,8,10,प्रयाणकाले मनसाऽचलेन\n\nभक्त्या युक्तो योगबलेन...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
313,8,9,कविं पुराणमनुशासितार\n\nमणोरणीयांसमनुस्मरेद्यः...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...


## Euclidean Distance

In [23]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [24]:
verse_index=mean_ed(query_embedding)

In [25]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
214,5,16,ज्ञानेन तु तदज्ञानं येषां नाशितमात्मनः।\n\nतेष...,रात्रि के अंधकार को मिटाने की सूर्य की शक्ति अ...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
503,13,21,कार्यकारणकर्तृत्वे हेतुः प्रकृतिरुच्यते।पुरुषः...,प्राकृत शक्ति ब्रह्मा के निर्देशानुसार असंख्य ...
548,15,4,ततः पदं तत्परिमार्गितव्य यस्मिन्गता न निवर्...,देहधारी जीवात्माएँ 'संसार' में डूबी रहती हैं अ...
547,15,3,न रूपमस्येह तथोपलभ्यते नान्तो न चादिर्न च स...,देहधारी जीवात्माएँ 'संसार' में डूबी रहती हैं अ...
313,8,9,कविं पुराणमनुशासितार\n\nमणोरणीयांसमनुस्मरेद्यः...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
314,8,10,प्रयाणकाले मनसाऽचलेन\n\nभक्त्या युक्तो योगबलेन...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
286,7,12,ये चैव सात्त्विका भावा राजसास्तामसाश्च ये।\n\n...,पिछले चार श्लोकों में अपनी महिमा का वर्णन करने...


# Line Matching

## Cosine Similarity

In [26]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [27]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
279,7,5,अपरेयमितस्त्वन्यां प्रकृतिं विद्धि मे पराम्।\n...,श्रीकृष्ण अब पूर्णतया भौतिक ज्ञान के क्षेत्र स...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
280,7,6,एतद्योनीनि भूतानि सर्वाणीत्युपधारय।\n\nअहं कृत...,भौतिक जगत के सभी प्राणी आत्मा और पदार्थ के संय...
288,7,14,दैवी ह्येषा गुणमयी मम माया दुरत्यया।\n\nमामेव ...,कुछ लोग यह स्वीकार करते हैं कि भौतिक शक्ति मिथ...
678,18,62,तमेव शरणं गच्छ सर्वभावेन भारत।तत्प्रसादात्परां...,भगवान पर निर्भरता के कारण आत्मा को अपनी वर्तमा...
331,8,27,नैते सृती पार्थ जानन्योगी मुह्यति कश्चन।\n\nतस...,जो साधक मन को भगवान में एकनिष्ठ करने का प्रयास...
41,1,47,सञ्जय उवाच\n\nएवमुक्त्वाऽर्जुनः संख्ये रथोपस्थ...,वार्तालाप करते समय मनुष्य प्रायः भावनाओं में ब...


## Euclidean Distance

In [28]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [29]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
309,8,5,अन्तकाले च मामेव स्मरन्मुक्त्वा कलेवरम्।\n\nयः...,अगले श्लोक में श्रीकृष्ण इस सिद्धान्त की व्या...
613,17,25,तदित्यनभिसन्धाय फलं यज्ञतपःक्रियाः।\n\nदानक्रि...,हमारे द्वारा किए जाने वाले कर्मों का फल देना भ...
430,11,22,रुद्रादित्या वसवो ये च साध्या\n\nविश्वेऽश्िवनौ...,ये सभी महान विभूतियाँ भगवान की शक्ति से ही अप...
64,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः।\n\nन...,"चेतना, जो आत्मा का लक्षण है, उसे भौतिक अवयवों..."
654,18,38,विषयेन्द्रियसंयोगाद्यत्तदग्रेऽमृतोपमम्।परिणामे...,राजसिक सुख एक रोमांच की भांति है जो इन्द्रियों...
457,11,49,मा ते व्यथा मा च विमूढभावो\n\nदृष्ट्वा रूपं घो...,श्रीकृष्ण निरन्तर अर्जुन को धीरज बंधाते हुए कह...
62,2,21,वेदाविनाशिनं नित्यं य एनमजमव्ययम्।\n\nकथं स पु...,आध्यात्मिक रूप से उन्नत आत्मा हमारे उस अहं को ...
607,17,19,मूढग्राहेणात्मनो यत्पीडया क्रियते तपः।परस्योत्...,मूढग्राहेणात्मनो शब्द ऐसे व्यक्तियों के लिए प्...
517,13,35,क्षेत्रक्षेत्रज्ञयोरेवमन्तरं ज्ञानचक्षुषा।भूतप...,अपनी प्रथागत शैली में अब तक श्रीकृष्ण ने जो कु...
272,6,45,प्रयत्नाद्यतमानस्तु योगी संशुद्धकिल्बिषः।\n\nअ...,पिछले जन्म के संचित अभ्यास आध्यात्मिक उन्नति क...


# Whole Part

## Cosine Similarity

In [30]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [31]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
322,8,18,अव्यक्ताद्व्यक्तयः सर्वाः प्रभवन्त्यहरागमे।\n\...,ब्रह्माण्ड के अद्भुत ब्रह्माण्डीय खेल में विभि...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
338,9,6,यथाऽऽकाशस्थितो नित्यं वायुः सर्वत्रगो महान्।\n...,श्रीकृष्ण ने चौथे श्लोक से छठे श्लोक तक 'मत्स्...
314,8,10,प्रयाणकाले मनसाऽचलेन\n\nभक्त्या युक्तो योगबलेन...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
313,8,9,कविं पुराणमनुशासितार\n\nमणोरणीयांसमनुस्मरेद्यः...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...


## Euclidean Distance

In [32]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [33]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
22,1,26,तत्रापश्यत्स्थितान्पार्थः पितृ़नथ पितामहान्।\n...,अर्जुन युद्धभूमि में अपने सभी सम्बंधियों को दे...
486,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,श्रीकृष्ण अब स्वयं कई प्रश्न खड़े करते हैं और ...
8,1,11,अयनेषु च सर्वेषु यथाभागमवस्थिताः।\n\nभीष्ममेवा...,दुर्योधन यह समझ गया था कि भीष्म पितामह मानसिक ...
2,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम्।\n\...,दुर्योधन एक कुशल कूटनीतिज्ञ के रूप में अपने से...
3,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि।\n\nयुयु...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
4,1,5,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्।\n\nप...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
5,1,6,युधामन्युश्च विक्रान्त उत्तमौजाश्च वीर्यवान्।\...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
76,2,35,भयाद्रणादुपरतं मंस्यन्ते त्वां महारथाः।\n\nयेष...,अर्जुन एक पराक्रमी योद्धा के रूप में विख्यात थ...
10,1,13,ततः शङ्खाश्च भेर्यश्च पणवानकगोमुखाः।\n\nसहसैवा...,युद्ध के लिए भीष्म पितामह के तीव्र उत्साह को द...
660,18,44,कृषिगौरक्ष्यवाणिज्यं वैश्यकर्म स्वभावजम्।परिचर...,वैश्यों में रजोगुण की प्रधानता के साथ तमोगुण म...


# Clustering

## Max Pooling

In [34]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [35]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [36]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
10,1,13,ततः शङ्खाश्च भेर्यश्च पणवानकगोमुखाः।\n\nसहसैवा...,युद्ध के लिए भीष्म पितामह के तीव्र उत्साह को द...
16,1,19,स घोषो धार्तराष्ट्राणां हृदयानि व्यदारयत्।\n\n...,आपके पुत्रों के हृदयों को विदीर्ण कर दिया। पाण...
34,1,38,यद्यप्येते न पश्यन्ति लोभोपहतचेतसः।\n\nकुलक्षय...,यद्यपि अर्जुन वृत्ति से क्षत्रिय योद्धा था फिर...
35,1,39,कथं न ज्ञेयमस्माभिः पापादस्मान्निवर्तितुम्।\n\...,यद्यपि अर्जुन वृत्ति से क्षत्रिय योद्धा था फिर...
37,1,41,अधर्माभिभवात्कृष्ण प्रदुष्यन्ति कुलस्त्रियः।\n...,वैदिक सभ्यता में समाज में स्त्रियों को उच्च स...
56,2,15,यं हि न व्यथयन्त्येते पुरुषं पुरुषर्षभ।\n\nसमद...,पिछले श्लोक में श्रीकृष्ण ने यह वर्णन किया था ...
85,2,44,भोगैश्वर्यप्रसक्तानां तयापहृतचेतसाम्।\n\nव्यवस...,वे मनुष्य जिनका मन इन्द्रिय सुखों के प्रति आसक...
89,2,48,योगस्थः कुरु कर्माणि सङ्गं त्यक्त्वा धनञ्जय।\n...,समता का भाव जो हमें सभी परिस्थितियों को शांतिप...
104,2,63,क्रोधाद्भवति संमोहः संमोहात्स्मृतिविभ्रमः।\n\n...,जैसे प्रात:काल की धुंध सूर्य की किरणों को ढक ल...
111,2,70,आपूर्यमाणमचलप्रतिष्ठं\n\nसमुद्रमापः प्रविशन्ति...,"विशाल समुद्र में, निरन्तर नदियों के जल के प्र..."


## Mean Pooling

In [37]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [38]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [39]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
256,6,29,सर्वभूतस्थमात्मानं सर्वभूतानि चात्मनि।\n\nईक्ष...,भारत में दीपावली के उत्सव में दुकानों में कई प...
280,7,6,एतद्योनीनि भूतानि सर्वाणीत्युपधारय।\n\nअहं कृत...,भौतिक जगत के सभी प्राणी आत्मा और पदार्थ के संय...
307,8,3,श्री भगवानुवाच\n\nअक्षरं ब्रह्म परमं स्वभावोऽध...,श्रीकृष्ण कहते हैं कि परम सत्ता को ब्रह्म कहा...
308,8,4,अधिभूतं क्षरो भावः पुरुषश्चाधिदैवतम्।\n\nअधियज...,"पाँच तत्त्वों-पृथ्वी, जल, अग्नि, वायु और आकाश..."
313,8,9,कविं पुराणमनुशासितार\n\nमणोरणीयांसमनुस्मरेद्यः...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
314,8,10,प्रयाणकाले मनसाऽचलेन\n\nभक्त्या युक्तो योगबलेन...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
322,8,18,अव्यक्ताद्व्यक्तयः सर्वाः प्रभवन्त्यहरागमे।\n\...,ब्रह्माण्ड के अद्भुत ब्रह्माण्डीय खेल में विभि...
323,8,19,भूतग्रामः स एवायं भूत्वा भूत्वा प्रलीयते।\n\nर...,वेदों में चार प्रकार की प्रलय का उल्लेख किया ग...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...


## Whole

In [40]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [41]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [42]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
216,5,18,विद्याविनयसंपन्ने ब्राह्मणे गवि हस्तिनि।\n\nशु...,जब हम ज्ञान के परिप्रेक्ष्य में चीजों को देखते...
278,7,4,भूमिरापोऽनलो वायुः खं मनो बुद्धिरेव च।\n\nअहङ्...,"प्राकृत शक्ति से रचित संसार विचित्र, जटिल और अ..."
286,7,12,ये चैव सात्त्विका भावा राजसास्तामसाश्च ये।\n\n...,पिछले चार श्लोकों में अपनी महिमा का वर्णन करने...
313,8,9,कविं पुराणमनुशासितार\n\nमणोरणीयांसमनुस्मरेद्यः...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
314,8,10,प्रयाणकाले मनसाऽचलेन\n\nभक्त्या युक्तो योगबलेन...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
400,10,34,मृत्युः सर्वहरश्चाहमुद्भवश्च भविष्यताम्।\n\nकी...,अंग्रेजी में एक कहावत है 'मृत्यु के समान अटल'...
411,11,3,एवमेतद्यथात्थ त्वमात्मानं परमेश्वर।\n\nद्रष्टु...,अर्जुन ने श्रीकृष्ण को पुरुषोत्तम कहकर संबोधित...
416,11,8,न तु मां शक्यसे द्रष्टुमनेनैव स्वचक्षुषा।\n\nद...,जब परमप्रभु संसार में अवतरित होते हैं तब उनके ...
